<a href="https://kritikseth.github.io/ipynbtagredirect" target="_parent"><img src="https://raw.githack.com/kritikseth/kritikseth/master/assets/icons/kritik_ipynbtagredirect.svg" alt="Kritik Seth"/></a>

If you are viewing this in GitHub and cannot see the plots, [Click Here](https://nbviewer.jupyter.org/github/kritikseth/ML-College-Sem-V/blob/master/ML_12_J043_121020.ipynb)

[Click Here](https://github.com/kritikseth/ML-College-Sem-V/blob/master/ML_12_J043_121020.ipynb) to go back to GitHub

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten

import plotly.express as px

In [2]:
!wget -O AirQualityUCI.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
!unzip AirQualityUCI.zip

--2020-10-29 19:20:39--  https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1543989 (1.5M) [application/x-httpd-php]
Saving to: ‘AirQualityUCI.zip’

AirQualityUCI.zip   100%[===================>]   1.47M  3.06MB/s    in 0.5s    

2020-10-29 19:20:39 (3.06 MB/s) - ‘AirQualityUCI.zip’ saved [1543989/1543989]

Archive:  AirQualityUCI.zip
  inflating: AirQualityUCI.csv       
  inflating: AirQualityUCI.xlsx      


In [3]:
df = pd.read_excel('/content/AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']])
df.replace(to_replace=-200.0, value=0.0, inplace=True)

In [4]:
wv = df['NMHC(GT)']
bad_wv = wv == -200.0
wv[bad_wv] = 0.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
target = 'NO2(GT)'
df.corr()[[target]]

,NO2(GT)
CO(GT),0.723154
PT08.S1(CO),0.284508
NMHC(GT),0.099541
C6H6(GT),0.402581
PT08.S2(NMHC),0.334108
NOx(GT),0.795888
PT08.S3(NOx),-0.440202
NO2(GT),1.000000
PT08.S4(NO2),0.010185
PT08.S5(O3),0.439057


In [6]:
df = df[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)','PT08.S2(NMHC)', 'NOx(GT)','NO2(GT)','PT08.S5(O3)']]

In [7]:
df.head()

,CO(GT),PT08.S1(CO),NMHC(GT),PT08.S2(NMHC),NOx(GT),NO2(GT),PT08.S5(O3)
0,2.6,1360.00,150,1045.50,166.0,113.0,1267.50
1,2.0,1292.25,112,954.75,103.0,92.0,972.25
2,2.2,1402.00,88,939.25,131.0,114.0,1074.00
3,2.2,1375.50,80,948.25,172.0,122.0,1203.25
4,1.6,1272.25,51,835.50,131.0,116.0,1110.00


In [8]:
df.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),PT08.S2(NMHC),NOx(GT),NO2(GT),PT08.S5(O3)
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,1.765545,1056.692672,21.373731,902.298983,203.636796,93.232617,982.774554
std,1.554264,301.232260,91.103489,318.681183,214.984126,61.468588,438.060087
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.600000,921.000000,0.000000,711.000000,50.000000,53.000000,699.750000
50%,1.500000,1052.500000,0.000000,894.500000,141.000000,96.000000,942.000000
75%,2.600000,1221.250000,0.000000,1104.750000,284.200000,133.000000,1255.250000
max,11.900000,2039.750000,1189.000000,2214.000000,1479.000000,339.700000,2522.750000


Log and Square Root Transformation

In [9]:
df['NOx(GT)'] = np.log1p(df['NOx(GT)'])
df['NMHC(GT)'] = np.log1p(df['NMHC(GT)'])
df['CO(GT)'] = np.sqrt(df['CO(GT)'])

In [10]:
X = df.drop(target, axis=1)
Y = df[[target]]

Scaling (0-1)

In [11]:
scaler = MinMaxScaler()

In [12]:
X = scaler.fit_transform(X)
Y = scaler.fit_transform(Y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7485, 6), (1872, 6), (7485, 1), (1872, 1))

In [15]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7485, 6, 1), (1872, 6, 1), (7485, 1), (1872, 1))

In [17]:
model = Sequential()

model.add(LSTM(units=40, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=60, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=40))
model.add(Dense(units=1))

In [18]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
234/234 [==============================] - 2s 6ms/step - loss: 0.0219 - accuracy: 0.1741
Epoch 2/20
234/234 [==============================] - 2s 7ms/step - loss: 0.0174 - accuracy: 0.1741
Epoch 3/20
234/234 [==============================] - 2s 7ms/step - loss: 0.0159 - accuracy: 0.1741
Epoch 4/20
234/234 [==============================] - 2s 7ms/step - loss: 0.0099 - accuracy: 0.1741
Epoch 5/20
234/234 [==============================] - 2s 7ms/step - loss: 0.0074 - accuracy: 0.1741
Epoch 6/20
234/234 [==============================] - 2s 6ms/step - loss: 0.0067 - accuracy: 0.1742
Epoch 7/20
234/234 [==============================] - 2s 7ms/step - loss: 0.0065 - accuracy: 0.1742
Epoch 8/20
234/234 [==============================] - 2s 7ms/step - loss: 0.0064 - accuracy: 0.1742
Epoch 9/20
234/234 [==============================] - 2s 6ms/step - loss: 0.0062 - accuracy: 0.1742
Epoch 10/20
234/234 [==============================] - 2s 7ms/step - loss: 0.0063 - accuracy: 0.1742

In [20]:
model.evaluate(X_test, y_test)

59/59 [==============================] - 0s 3ms/step - loss: 0.0054 - accuracy: 0.1811


[0.0054321205243468285, 0.18108974397182465]

In [21]:
y_pred = model.predict(X_test).tolist()
y_pred = [item for sublist in y_pred for item in sublist]

In [22]:
fig = px.line(y=y_pred)

fig.show()